In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import folium
import os
from shapely import wkt
from folium.plugins import Geocoder
from folium import FeatureGroup, LayerControl

# Get the city from command-line arguments
city = os.environ.get('CITY', 'Default City Name')
base_path = "/Users/leonardo/Desktop/Tesi/LTSBikePlan/data/"

# Sanitize the city name
city_sanitized = city.split(",")[0].replace(" ", "_")
city_path = f"{base_path}{city_sanitized}"  # Use the sanitized city name here

# Load data
all_lts_df = pd.read_csv(f"{city_path}_all_lts.csv")
#all_lts_df = pd.read_csv("/Users/leonardo/Desktop/Tesi/LTSBikePlan/data/Montereale_Valcellina_all_lts.csv")
# Convert 'geometry' column from WKT strings to actual geometry objects
all_lts_df['geometry'] = all_lts_df['geometry'].apply(wkt.loads)

# Convert the DataFrame to a GeoDataFrame
all_lts = gpd.GeoDataFrame(all_lts_df, geometry='geometry')

# Set the CRS for the GeoDataFrame
all_lts.crs = "EPSG:32632"

# Reproject to WGS84
all_lts_projected = all_lts.to_crs(epsg=4326)

gdf_nodes = pd.read_csv(f"{city_path}_gdf_nodes.csv", index_col=0)
#gdf_nodes = pd.read_csv("/Users/leonardo/Desktop/Tesi/LTSBikePlan/data/Montereale_Valcellina_gdf_nodes.csv", index_col=0)

# Create a colormap for slope classes
color_palette = ["forestgreen", "dodgerblue", "#f4e800", "firebrick", "#000000", "purple"]
lts_classes = [1, 2, 3, 4, "undefined", "not cyclable"]
colors = dict(zip(lts_classes, color_palette))

# Calculate the mean of latitudes and longitudes
mean_latitude = all_lts_projected.geometry.apply(lambda geom: geom.centroid.y).mean()
mean_longitude = all_lts_projected.geometry.apply(lambda geom: geom.centroid.x).mean()

# Create a folium map centered on the mean of latitudes and longitudes
map_osm = folium.Map(location=[mean_latitude, mean_longitude], zoom_start=11.5)

# Add a general search address functionality
geocoder = Geocoder(
    position='topleft',
    control=True,
    collapsed=True,
    auto_type=True,
    placeholder='Search for a location...',
    max_zoom=15,
    hide_marker=True
).add_to(map_osm)

# Create a FeatureGroup for each stress level and add them to the map
feature_groups = {}

for lts_class, color in colors.items():
    if isinstance(lts_class, str):  # For "Undefined" and "Not cyclable"
        name = lts_class
    else:
        name = f"LTS {lts_class}"
    toggle_switch_html = f'''
    <div class="toggle-switch active" onclick="toggleSwitch(this)" style="background-color: {color};"></div>
    <span>{name}</span>
    '''
    feature_group = FeatureGroup(name=toggle_switch_html, show=True)
    feature_groups[lts_class] = feature_group
    feature_group.add_to(map_osm)

# Add stress information to the map using the feature groups
for _, row in all_lts_projected.iterrows():
    if pd.isna(row['name']):
        row_name = ''
    else:
        row_name = row['name'].lower()  # Convert name to lowercase for easier matching

    color = "#000000"  # Default color
    excluded_terms = ["bici", "cicla", "ciclo"]
    if any(keyword in row_name for keyword in ["tangenziale", "superstrada", "strada statale", "autostrada", "strada a scorrimento veloce"]) and not any(term in row_name for term in excluded_terms):
        lts_class = "not cyclable"
        color = colors[lts_class]
    elif row['lts'] in feature_groups:
        lts_class = row['lts']
        color = colors[lts_class]
    else:
        lts_class = "undefined"

    folium.GeoJson(
        row['geometry'], 
        style_function=lambda _, color=color: {'color': color}
    ).add_to(feature_groups[lts_class])

# Add layer control to map
LayerControl(position='topright').add_to(map_osm)

# Add the custom header to the map
map_html = open("custom_map.html", "r").read()
map_osm.get_root().html.add_child(folium.Element(map_html))

# Save the map
map_osm.save(f"{city_path}_lts_map.html")

# Display the map 
map_osm